# Semantic segmentation test

---

**Author:** [rodoart](https://github.com/rodoart/)<br>
**Date created:** 2021/07/24<br>
**Last modified:** 2021/07/24<br>
**Description:** 
Using pretrained neural networks to segment images of rooms.

# Semantic segmentation of rooms

## Path config

If you want the files to be copied to another folder within the same machine you are working on, by a source path other than remote.

In [1]:
PROJECT_SLUG = 'vigilancia_mascotas'
NAME = 'semantic_segmentation_test_with_room_data'
NUMBER = '0.1'


NOTEBOOK_NAME = f'{NUMBER}-{PROJECT_SLUG}-{NAME}.ipynb'

# COLAB
#DRIVE_MOUNT = '/drive'
#REMOTE_PATH = f'{DRIVE_MOUNT}/MyDrive/proyects/{PROJECT_SLUG}'
#LOCAL_PATH = '.'
#NOTEBOOK_PATH = f'{DRIVE_MOUNT}/MyDrive/Colab Notebooks/{NOTEBOOK_NAME}'

# LOCAL
REMOTE_PATH = '..'
LOCAL_PATH = '..'
DRIVE_MOUNT = ''
NOTEBOOK_PATH = f'G:\\Mi unidad\\Colab Notebooks\\{NOTEBOOK_NAME}'

In [2]:
if DRIVE_MOUNT:
    from google.colab import drive
    drive.mount(DRIVE_MOUNT)
    to_remote = True
else:
    to_remote = False

## Path functions

In [3]:
import sys

# It depends on where the library that comes with this package is stored.
sys.path.append(REMOTE_PATH)

In [4]:
from vigilancia_mascotas.utils.paths \
    import make_remote_copy_of_workspace_functions

In [5]:
local_dir, update_from_remote, update_to_remote, update_notebook = \
    make_remote_copy_of_workspace_functions(
        local_path=LOCAL_PATH,
        remote_path=REMOTE_PATH,
        notebook_path = NOTEBOOK_PATH
    )

## Update local folders

In [6]:
update_from_remote()

There is no need to copy the remote files, as the remote repository does not exist or is empty.


## Prepare the dataset

### Download

For the development of this experiment, the data set of Unity Computer Vision Datasets (Home interior sample) is used, which contains 1,000 artificially created with different types of labeling, including semantic segmentation.

![Example from the dataset](https://content.cdntwrk.com/files/aHViPTEwODAwMiZjbWQ9aXRlbWVkaXRvcmltYWdlJmZpbGVuYW1lPWl0ZW1lZGl0b3JpbWFnZV82MGY5YmY3MGYxNmZkLnBuZyZ2ZXJzaW9uPTAwMDAmc2lnPTIzZTcwMzIwNzIzMGIyMzAwNmZkM2VhNGZiODFiYzkz)

Access to the dataset can be obtained directly on the provided page. The following download section is designed to work only for contributors to this project, but can be easily modified once you get the download link.

In [7]:
import requests

In [8]:
file_name = "Unity_Residential_Interiors.zip"

url_dataset_zip = "https://storage.googleapis.com/computer-vision-datasets/"\
                + f"residential/{file_name}"

zip_destiny_path = local_dir('tmp', file_name)

In [9]:
if not zip_destiny_path.is_file():
    response = requests.get(URL)
    open(zip_destiny_path, "wb").write(response.content)

### Unzip the files

In [10]:
import zipfile
from vigilancia_mascotas.utils.paths import is_valid

In [11]:
unzip_path = local_dir('tmp', file_name[:-4].lower())

In [12]:
if not is_valid(unzip_path):
    with zipfile.ZipFile(zip_destiny_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_path)


### Moving files

In [29]:
import shutil
from os import makedirs

In [30]:
dataset_path = local_dir('data','raw','semantic_segmentation', 'unity_residential_interiors')

makedirs(dataset_path, exist_ok=True)

In [33]:
tmp_images_dir = unzip_path.joinpath(
    'Residential Interiors Dataset',
    'RGB2d9da855-d495-4bd9-9a1f-6744db5a3249')

raw_images_dir = dataset_path.joinpath('images')

tmp_labels_dir = tmp_images_dir.parent.joinpath(
    'SemanticSegmentation50529ca2-588c-4aba-ab92-a9d4a31a56d4'
)
raw_labels_dir = dataset_path.joinpath('labels')

In [35]:
for source, destiny in zip([tmp_images_dir, tmp_labels_dir], 
                           [raw_images_dir, raw_labels_dir]):
    if not is_valid(destiny):
        shutil.move(
            src=source,
            dst=destiny
        )

### Separate train and test subsets

Shuffle file names

In [37]:
from os import listdir

In [57]:
# Obtain file_names
file_names = [name[4:-4] for name in listdir(raw_images_dir) if name.endswith('.png')]
len(file_names)

1000

In [49]:
from random import shuffle

In [58]:
shuffle(file_names)

In [61]:
n_train = round(len(file_names)*0.7)

train_names = file_names[:n_train]
val_names = file_names[n_train:]

### Move files to train and test folders

In [69]:
dataset_processed_path = local_dir('data','processed','semantic_segmentation', 'unity_residential_interiors')

In [71]:
dataset_path

WindowsPath('G:/Mi unidad/IA/seminario_innovacion/vigilancia_mascotas/data/raw/semantic_segmentation/unity_residential_interiors')

In [100]:
from os import mkdir
from os import rmdir

In [107]:
if not is_valid(dataset_processed_path):

    dataset_processed_path.mkdir(parents=True, exist_ok=True)

    for set_name, file_names in zip(['test', 'val'], [train_names, val_names]):
        for set_type, name_prefix in zip(['images', 'labels'], 
                                        ['rgb', 'segmentation']):
            for file_name in file_names:
                destiny_path = dataset_processed_path.joinpath(
                    f'{set_name}_{set_type}',
                    f'{file_name}.png'
                )

                origin_path = dataset_path.joinpath(
                    set_type, f'{name_prefix}_{file_name}.png'
                )

                if not destiny_path.parent.is_dir():
                    destiny_path.parent.mkdir(exist_ok=True)

                shutil.copy(origin_path, destiny_path)

## Update remote folders

In [ ]:
update_notebook(to_remote)

The notebook 0.0-vigilancia_mascotas-semantic_segmentation_test.ipynb has been updated in the remote folder


In [ ]:
update_to_remote()